In [ ]:
using Pkg

Pkg.activate("/home/rs/.julia/environments/triangulations/")

In [ ]:
using Random
Random.seed!(1);

# CairoMakie plotting basics

In [ ]:
using AbstractPlotting
using CairoMakie

In [ ]:
n = 7

x = 400 * rand(n)
y = 200 * rand(n)
[x y]'

In [ ]:
scene = Scene(resolution=(400,200), show_axis=false, scale_plot=false)
scatter!(x, y, markersize=5)
linesegments!(x, y)

In [ ]:
make_scene() = Scene(resolution=(400,200), show_axis=false, scale_plot=false)

# Triangle.jl

In [ ]:
using Triangle

In [ ]:
# Start with straight-forward Delaunay triangulation, again.
points = [x y]
points_map = collect(1:n)

triangles = basic_triangulation(points, points_map)
@show triangles

edges = reshape(vcat([t[[1, 2, 2, 3, 3, 1]] for t in triangles]...), 2, :)
@show edges

scene = Scene(resolution=(600,300))
scatter!(x, y, markersize=8, color=:red)
linesegments!(points[vec(edges), :], linewidth=2)

In [ ]:
using Statistics

In [ ]:
# for each triangle, add a point in the center (mean)
centers = [mean(points[t, :], dims=1) for t in triangles]

In [ ]:
centers_array = reshape(hcat(centers...), 2, :)'
@show centers_array
scatter!(centers_array, markersize=5)

In [ ]:
edges'[:,:]

In [ ]:
points2 = vcat(points, centers_array)
points_map2 = collect(1:size(points2, 1))
triangles2 = constrained_triangulation(points2, points_map2, edges'[:,:])

edges2 = reshape(vcat([t[[1, 2, 2, 3, 3, 1]] for t in triangles2]...), 2, :)
linesegments!(points2[vec(edges2), :], color=:green)

In [ ]:
# need also subdivide the edges that we want to keep?
# TODO...